In [166]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.plotting import figure,show
from bokeh.io import output_notebook, gridplot, output_file, show, save, vplot
import struct
import datetime

In [54]:
import os
import pandas as pd

In [291]:
import scipy.io as sio

In [ ]:
file_path = './636151800793559606/ChGroup_13/SpikeTimes'

In [ ]:

file = open(file_path, "rb")

spikes = np.fromfile(file, dtype=np.uint64)

file.close() 

fs=30e3
MSDNrate=1e7


ephysfilename = 636151800793559606

spike_times = [thing*MSDNrate/fs + np.uint64(ephysfilename) for thing in spikes]


In [ ]:
len(spike_times)

In [ ]:
spike_times[0:10]

In [25]:
######### ! Convert the matlab timestamp to actual time:
def matlab2datetime(matlab_datenum):
    # input matlab_datenum = e.g. 636150871373353768
    #matlab_datenum = int(matlab_datenum)
    matlab_datenum = matlab_datenum/(24*3600*1e7)+367
    
    day = datetime.datetime.fromordinal(int(matlab_datenum))
    #print 'day = ', day
    dayfrac = datetime.timedelta(days=matlab_datenum%1) - datetime.timedelta(days = 366)
    #print 'dayfrac = ', dayfrac
    return day + dayfrac

In [26]:
spike_time_1 = matlab2datetime(spike_times[0])

In [27]:
start_time = matlab2datetime(int(ephysfilename))

In [359]:
spike_seconds = [(matlab2datetime(thing) - start_time).total_seconds() for thing in spike_times] # convert 636s to datetime objects

In [360]:
spike_seconds = np.array(spike_seconds)

In [363]:
spike_seconds.shape

(1393389,)

In [361]:
spike_seconds[0:20]

array([ 0.101076,  0.103631,  0.113397,  0.114976,  0.118105,  0.120166,
        0.124763,  0.126775,  0.127942,  0.129702,  0.134801,  0.138573,
        0.1398  ,  0.140866,  0.142938,  0.14498 ,  0.148078,  0.151066,
        0.152544,  0.153731])

In [190]:
#### get the TRIAL TIMES CSV file ######
stim_data = pd.read_csv('oe_stim_times.csv') # read the CSV file with stimulus times and orientations

In [191]:
stim_data

,Unnamed: 0,orientations,times
0,0,18,503.748688
1,1,144,530.972582
2,2,144,577.723023
3,3,72,588.452629
4,4,54,609.062156
5,5,0,629.971556
6,6,126,640.917733
7,7,162,650.431186
8,8,90,769.123284
9,9,144,779.269747


In [206]:
trials = range(len(stim_data))

In [198]:
spikes = []

In [358]:
for i in trials:

    trial_time = stim_data.times[i]
    trial_range = 0.5
    temp_idx = (spike_seconds < trial_time+trial_range) & (spike_seconds > trial_time - trial_range)
    
    print 'temp_idx = ', temp_idx
    temp_trial = spike_seconds[temp_idx] - trial_time
    
    #print temp_trial
    
    spikes.append(temp_trial)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [200]:
len(spikes)

91

In [201]:
stim_data['spikes'] = spikes

In [202]:
stim_data

,Unnamed: 0,orientations,times,spikes
0,0,18,503.748688,"[-0.499185073, -0.497776073, -0.495926073, -0...."
1,1,144,530.972582,"[-0.496441303, -0.493906303, -0.491251303, -0...."
2,2,144,577.723023,"[-0.498482524, -0.495113524, -0.491150524, -0...."
3,3,72,588.452629,"[-0.4990691, -0.4976911, -0.4961621, -0.491233..."
4,4,54,609.062156,"[-0.496184568, -0.494756568, -0.493317568, -0...."
5,5,0,629.971556,"[-0.49658563, -0.49545863, -0.49331663, -0.490..."
6,6,126,640.917733,"[-0.499631301, -0.498565301, -0.496594301, -0...."
7,7,162,650.431186,"[-0.499647801, -0.495221801, -0.490282801, -0...."
8,8,90,769.123284,"[-0.499320116, -0.497177116, -0.495045116, -0...."
9,9,144,779.269747,"[-0.499610204, -0.498010204, -0.495345204, -0...."


In [90]:
stim_data.to_csv('stimANDspike_times.csv')

In [91]:
###### organize the dataframe by orientation of the stimulus:


stim_orientation,idx,inv = np.unique(stim_data.orientations,return_index=True,return_inverse=True)

In [106]:
stim_dict = dict()
for ori in stim_orientation:	
    stim_dict[str(ori)] = stim_data.orientations[stim_data.orientations==ori].index.values ### put indices of stim_data that correspond to this orientation, from inv
    #### ^ stim_dict gives indices of stim_data for given orientations. 

In [110]:
stim_dict

{'0': array([ 5, 18, 32, 50, 57, 67, 75, 85]),
 '108': array([19, 21, 25, 33, 40, 53, 70]),
 '126': array([ 6, 12, 14, 31, 36, 39, 51, 55, 56, 69, 78]),
 '144': array([ 1,  2,  9, 43, 58, 61, 74, 76, 81, 87]),
 '162': array([ 7, 11, 17, 26, 34, 44, 49, 72, 89]),
 '18': array([ 0, 13, 15, 24, 37, 62, 79, 82]),
 '36': array([29, 38, 46, 65, 66, 77, 83, 86]),
 '54': array([ 4, 10, 16, 23, 28, 48, 63, 84]),
 '72': array([ 3, 20, 41, 52, 59, 71, 80, 88]),
 '90': array([ 8, 22, 27, 30, 35, 42, 45, 47, 54, 60, 64, 68, 73, 90])}

In [203]:
for ori in stim_dict:
    #print thing, stim_dict[thing]
    print 'Orientation %s' % ori
    print stim_data.spikes[stim_dict[ori]]
    ##### stim_dict[ori] is an array of trial numbers for a particular orientation

Orientation 144
1     [-0.496441303, -0.493906303, -0.491251303, -0....
2     [-0.498482524, -0.495113524, -0.491150524, -0....
9     [-0.499610204, -0.498010204, -0.495345204, -0....
43    [-0.49988253, -0.49641253, -0.49450053, -0.493...
58    [-0.49756775, -0.49413775, -0.49082875, -0.488...
61    [-0.49537294, -0.49236594, -0.49153094, -0.486...
74    [-0.49778545, -0.49448645, -0.48992945, -0.487...
76    [-0.48849823, -0.48730123, -0.48584323, -0.482...
81    [-0.49703252, -0.49559352, -0.49135952, -0.489...
87    [-0.49916106, -0.49773306, -0.49505706, -0.493...
Name: spikes, dtype: object
Orientation 54
4     [-0.496184568, -0.494756568, -0.493317568, -0....
10    [-0.497694126, -0.496457126, -0.494757126, -0....
16    [-0.497529361, -0.494722361, -0.492670361, -0....
23    [-0.49806106, -0.49607006, -0.49309206, -0.492...
28    [-0.49898105, -0.49722105, -0.49605405, -0.489...
48    [-0.4994558, -0.4972328, -0.4958248, -0.494657...
63    [-0.49866657, -0.49680557, -0.49300357,

In [170]:
save_folder = './unsortedPSTH_channel_13'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)


In [289]:
range(len(stim_data.spikes[stim_dict['0']]))

[0, 1, 2, 3, 4, 5, 6, 7]

In [290]:
for ori in stim_dict:
    
    print stim_data.spikes[stim_dict[ori]]

    raster = figure(width=1000, height=400,y_axis_label='Trial Number',title='Raster')
    all_spike_times = []
    
    num_trials_ori = range(len(stim_data.spikes[stim_dict[ori]]))
    
    for trial in num_trials_ori:
        print trial
        trial_time = stim_data.spikes[stim_dict[ori][trial]] ## spike times on this trial
        num_spikes_ontrial = len(trial_time)
        
        all_spike_times.append(trial_time)
    
        ####### MAKE RASTER
        raster.segment(x0=trial_time, y0=np.repeat(trial,num_spikes_ontrial), x1=trial_time,
                       y1=np.repeat(trial+1,num_spikes_ontrial), color="black", line_width=0.5)
        
    
    
    
    histo_fig = figure(width=1000, plot_height=500,y_axis_label='Firing Rate (Hz)',x_axis_label='Time (sec)',x_range=raster.x_range)

    num_trials = len(trials)
    bin_time = 0.025 # 25 ms bins
    num_bins = 1.0 / bin_time
    time_vec = np.linspace(-0.5,0.5,num_bins)


    
    all_spike_times = np.concatenate(all_spike_times).ravel() ## make one long vector of spike times. 

    hist, edges = np.histogram(all_spike_times, bins=num_bins)
    bin_width = np.diff(edges)[0] # in seconds.
    histo_fig.quad(top=hist/bin_width/num_trials, bottom=0, left=edges[:-1], right=edges[1:], ## hist/bin_width = firing rate in Hz
        fill_color="#036564", line_color="#033649")



    win_size = 2**4
    win_step = 2**2

    #print np.int16(all_spike_times[0:2] * 3e4)

    ##################!!!!!!!!!!!###### -- should "spike_vec" here be the a sorted "all_spike_times"? yes... 
    win_x,windowd_spike_vec = windowed_histogram(np.sort(all_spike_times),time_vec,win_size,win_step)
    ##################!!

    all_spike_times = None 
    
    #histo_fig.line(win_x,[w/win_size/len(stim_dict[ori]) for w in windowd_spike_vec],color='magenta')

    output_file(save_folder + '/spike_histogram_'+str(ori)+'.html')
    grid = gridplot([[raster], [histo_fig]])

    save(grid)
    
    #stim_name = 'orientation_' + ori
    #output_file(save_folder + '/psth_'+str(stim_name)+'.html')
    #save(raster)

INFO:bokeh.core.state:Session output file './unsortedPSTH_channel_13/spike_histogram_144.html' already exists, will be overwritten.


1     [-0.496441303, -0.493906303, -0.491251303, -0....
2     [-0.498482524, -0.495113524, -0.491150524, -0....
9     [-0.499610204, -0.498010204, -0.495345204, -0....
43    [-0.49988253, -0.49641253, -0.49450053, -0.493...
58    [-0.49756775, -0.49413775, -0.49082875, -0.488...
61    [-0.49537294, -0.49236594, -0.49153094, -0.486...
74    [-0.49778545, -0.49448645, -0.48992945, -0.487...
76    [-0.48849823, -0.48730123, -0.48584323, -0.482...
81    [-0.49703252, -0.49559352, -0.49135952, -0.489...
87    [-0.49916106, -0.49773306, -0.49505706, -0.493...
Name: spikes, dtype: object
0
1
2
3
4
5
6
7
8
9
-14996


INFO:bokeh.core.state:Session output file './unsortedPSTH_channel_13/spike_histogram_54.html' already exists, will be overwritten.


4     [-0.496184568, -0.494756568, -0.493317568, -0....
10    [-0.497694126, -0.496457126, -0.494757126, -0....
16    [-0.497529361, -0.494722361, -0.492670361, -0....
23    [-0.49806106, -0.49607006, -0.49309206, -0.492...
28    [-0.49898105, -0.49722105, -0.49605405, -0.489...
48    [-0.4994558, -0.4972328, -0.4958248, -0.494657...
63    [-0.49866657, -0.49680557, -0.49300357, -0.491...
84    [-0.49903726, -0.49488326, -0.49287226, -0.491...
Name: spikes, dtype: object
0
1
2
3
4
5
6
7
-14983


INFO:bokeh.core.state:Session output file './unsortedPSTH_channel_13/spike_histogram_36.html' already exists, will be overwritten.


...


4
5
6
7
-14995
8     [-0.499320116, -0.497177116, -0.495045116, -0....
22    [-0.4899317, -0.4873977, -0.4862307, -0.479461...
27    [-0.49637372, -0.49506572, -0.49246072, -0.490...
30    [-0.49812609, -0.49529009, -0.49312709, -0.491...
35    [-0.49936688, -0.49477088, -0.49193388, -0.486...
42    [-0.49135581, -0.48839881, -0.48713081, -0.486...
45    [-0.4979616, -0.4957896, -0.4935666, -0.490156...
47    [-0.49561619, -0.49119019, -0.48965119, -0.487...
54    [-0.49817251, -0.49514451, -0.48918051, -0.484...
60    [-0.49406702, -0.49287002, -0.49103902, -0.488...
64    [-0.4987167, -0.4955787, -0.4848157, -0.479645...
68    [-0.49927558, -0.49797758, -0.49627758, -0.493...
73    [-0.4983094, -0.4971024, -0.4909164, -0.487466...
90    [-0.49912589, -0.49659089, -0.49363389, -0.492...
Name: spikes, dtype: object
0
1
2
3
4
5
6


INFO:bokeh.core.state:Session output file './unsortedPSTH_channel_13/spike_histogram_90.html' already exists, will be overwritten.


7
8
9
10
11
12
13
-14981
7     [-0.499647801, -0.495221801, -0.490282801, -0....
11    [-0.499849835, -0.498854835, -0.496691835, -0....
17    [-0.499573782, -0.498014782, -0.492904782, -0....
26    [-0.49967382, -0.49833682, -0.49441382, -0.492...
34    [-0.49860846, -0.49440446, -0.49320746, -0.491...
44    [-0.4998447, -0.4983267, -0.4950167, -0.493880...
49    [-0.49692863, -0.49446463, -0.49333863, -0.490...
72    [-0.49991558, -0.49684758, -0.49457458, -0.493...
89    [-0.49704676, -0.49210776, -0.49104176, -0.488...
Name: spikes, dtype: object
0


INFO:bokeh.core.state:Session output file './unsortedPSTH_channel_13/spike_histogram_162.html' already exists, will be overwritten.


1
2
3
4
5
6
7
8
-14997


In [273]:
def slidingWindow(sequence,winSize,step=1):
    """Returns a generator that will iterate through
    the defined chunks of input sequence.  Input sequence
    must be iterable."""
 
    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # Pre-compute number of chunks to emit
    numOfChunks = ((len(sequence)-winSize)/step)+1
 
    # Do the work
    for i in range(0,numOfChunks*step,step):
        yield sequence[i:i+winSize]

def windowed_histogram(trace,time,window_size,step): ## for spike histogram, trace should be a 0's and 1's vector with length = trial length (1's in places where spikes occur)
    out_x = []              ## time = vector in samples. 
    out_y = []

    fs = 3e4
    trace = np.int16(trace * fs)
    print trace[0]
    time = np.int16(time * fs)
    
    y = slidingWindow(trace,window_size,step) #window_size/5
    x = slidingWindow(time,window_size,step) ### get centers of the time
    for value in y:
        out_y.append(np.sum(value)) ## take the number of spikes in this window
        
    for t in x:
        out_x.append(np.median(t)) ## take the median time, i.e. the center of the time window
    
    return out_x,out_y

32602

In [ ]:
###### GETTING CLUSTER INFO FROM behavior_clusters.mat file. 

In [292]:
clusters_file = './ChGroup13/behavior_clusters.mat'

In [293]:
clusters = sio.loadmat(clusters_file)

In [302]:
clusters['behavior_clusters'].dtype

dtype([('num', 'O'), ('times', 'O'), ('amps', 'O'), ('wv', 'O'), ('channel', 'O'), ('trueNumber', 'O'), ('timeBegin', 'O'), ('isi', 'O'), ('names', 'O'), ('firstFile', 'O'), ('color', 'O'), ('features', 'O')])

In [310]:
cluster_times = clusters['behavior_clusters']['times']

In [312]:
cluster_times[0]

array([ array([[  114.38714973],
       [  196.25024973],
       [  301.51164973],
       [  331.70994973],
       [  357.59384973],
       [  419.4906164 ],
       [  458.42174973],
       [  568.34198307],
       [  618.53584973],
       [  699.5087164 ],
       [  921.13898307],
       [ 1012.98608307]])], dtype=object)

In [443]:
clusters['behavior_clusters']['num'][2][0][0][0]

504

In [320]:
num_clusters = len(clusters['behavior_clusters'])


46

In [456]:
###### now that we have the times of the spikes in each cluster (46 clusters), 
###### let's loop over the behavior trials and each cluster to see if any cluster has spikes within the trial range

# make variable 'sorted_spikes' num_trials x 1. 

# for each behavior trial, add dictionary to sorted_spikes with name=cluster_number and entries = spike times.
trial_range = 0.5
rel_cluster_time = dict()
sorted_spikes = []

for trial in trials:
    #trial=0
    trial_time = stim_data.times[trial] # the time of the visual stimulus on this trial. 

    print 'Trial %d ranges from %5.3f to %5.3f seconds' % (trial, trial_time-trial_range, trial_time+trial_range)

    for cluster in range(num_clusters):
        #cluster = 0
        print 'cluster = ', cluster
        spikeSUA_seconds = clusters['behavior_clusters']['times'][cluster]

        spikeSUA_seconds = [x[0] for x in spikeSUA_seconds[0]] ### spikeSUA_seconds is an array of arrays. Unpack it!

        print 'spikeSUA_seconds = ', spikeSUA_seconds
        temp_idx = (spikeSUA_seconds < trial_time+trial_range) & (spikeSUA_seconds > trial_time - trial_range)


        

        temp_trial = spike_seconds[temp_idx] - trial_time ## subtract trial time so the spike time is in [-0.5 to +0.5]


        # rel_cluster_time will be our dict with keys=clustername, entries=times for the spike for this trial. 
        rel_cluster_time[str(clusters['behavior_clusters']['num'][cluster][0][0][0])] = temp_trial

    sorted_spikes.append(rel_cluster_time)

#print temp_trial

#spikes.append(temp_trial)

Trial 0 ranges from 503.249 to 504.249 seconds
cluster =  0
spikeSUA_seconds =  [114.38714973302558, 196.2502497327514, 301.51164973294362, 331.70994973322377, 357.59384973300621, 419.49061639979482, 458.42174973292276, 568.3419830664061, 618.53584973281249, 699.50871639978141, 921.13898306619376, 114.38714973302558]
cluster =  1
spikeSUA_seconds =  [36.473016399890184, 399.01048306608573, 426.18961639981717, 916.41724973265082]
cluster =  2
spikeSUA_seconds =  [302.0542163993232, 305.43251639977098, 309.61368306633085, 355.7033830662258, 460.79344973294064, 466.50381639972329, 505.0494497329928, 521.56914973305538, 523.73698306642473, 623.95691639976576, 749.44448306644335, 964.63818306615576, 1082.2294830665924, 1191.3241497329436]
cluster =  3
spikeSUA_seconds =  [722.0498163998127, 908.00138306617737, 966.83484973292798, 1176.0723163997754, 1179.8302830662578, 1316.5058163995855]
cluster =  4
spikeSUA_seconds =  [318.40328306658193, 409.7825830662623, 474.71411639964208, 746.172583

/Users/guitchounts/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1393389 but corresponding boolean dimension is 12
/Users/guitchounts/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1393389 but corresponding boolean dimension is 4
/Users/guitchounts/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1393389 but corresponding boolean dimension is 14
/Users/guitchounts/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:30: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 1393389 but corresponding boolean dimension is 6
/Users/guitchounts/anaconda/lib/python2.7/site-packages/ipykernel/__ma

 [318.40328306658193, 409.7825830662623, 474.71411639964208, 746.17258306639269, 899.82221639947966, 915.40044973324984, 928.43578306632116, 932.47858306625858, 1005.212716399692, 1009.6387830660678, 1012.680249732919, 1022.2404830665328, 1052.902149733156, 1071.7345497333445, 1072.7002163995057, 1103.0507497331128, 1156.5102830659598, 1254.238616399467, 1321.8280497328378, 1362.2472163992934, 1364.7615163996816, 1417.3573163994588, 1504.0171163994819, 1602.89924973296, 1621.2182830660604, 1707.3281830660999]
cluster =  5
spikeSUA_seconds =  [624.94698306638747, 696.43941639969125, 752.11441639997065, 1070.6475497330539, 1076.6020497330464, 1146.7530830660835, 1227.6949497330934, 1363.3923497330397]
cluster =  6
spikeSUA_seconds =  [1122.6195830665529, 1191.6398830660619, 1523.0657163998112, 1707.7948163999245]
cluster =  7
spikeSUA_seconds =  [1082.2367830662988, 1102.2426830665208, 1106.3023163997568, 1112.1621163995005, 1124.6760830665007, 1128.239416399505, 1133.4336830666289, 1140

In [458]:
sorted_spikes

[{'502': array([], dtype=float64),
  '503': array([], dtype=float64),
  '504': array([], dtype=float64),
  '505': array([], dtype=float64),
  '506': array([], dtype=float64),
  '507': array([], dtype=float64),
  '508': array([], dtype=float64),
  '509': array([], dtype=float64),
  '510': array([], dtype=float64),
  '511': array([], dtype=float64),
  '512': array([], dtype=float64),
  '513': array([], dtype=float64),
  '514': array([], dtype=float64),
  '515': array([], dtype=float64),
  '516': array([], dtype=float64),
  '517': array([], dtype=float64),
  '518': array([], dtype=float64),
  '519': array([], dtype=float64),
  '520': array([], dtype=float64),
  '521': array([], dtype=float64),
  '522': array([], dtype=float64),
  '523': array([], dtype=float64),
  '524': array([], dtype=float64),
  '525': array([], dtype=float64),
  '526': array([], dtype=float64),
  '527': array([], dtype=float64),
  '528': array([], dtype=float64),
  '529': array([], dtype=float64),
  '530': array([], d

In [429]:
xx = [x[0] for x in spikeSUA_seconds[0]]

In [430]:
xx

[114.38714973302558,
 196.2502497327514,
 301.51164973294362,
 331.70994973322377,
 357.59384973300621,
 419.49061639979482,
 458.42174973292276,
 568.3419830664061,
 618.53584973281249,
 699.50871639978141,
 921.13898306619376,
 114.38714973302558]

In [375]:
spikeSUA_seconds

array([ array([[ 3332.9403164 ],
       [ 3405.54264973],
       [ 3423.15874973],
       [ 3432.55324973],
       [ 3511.20938307],
       [ 3785.5665164 ],
       [ 3794.05364973],
       [ 3842.86504973]])], dtype=object)

In [459]:
print 577.723023+0.5

578.223023


In [316]:
stim_data

,Unnamed: 0,orientations,times,spikes
0,0,18,503.748688,"[-0.499185073, -0.497776073, -0.495926073, -0...."
1,1,144,530.972582,"[-0.496441303, -0.493906303, -0.491251303, -0...."
2,2,144,577.723023,"[-0.498482524, -0.495113524, -0.491150524, -0...."
3,3,72,588.452629,"[-0.4990691, -0.4976911, -0.4961621, -0.491233..."
4,4,54,609.062156,"[-0.496184568, -0.494756568, -0.493317568, -0...."
5,5,0,629.971556,"[-0.49658563, -0.49545863, -0.49331663, -0.490..."
6,6,126,640.917733,"[-0.499631301, -0.498565301, -0.496594301, -0...."
7,7,162,650.431186,"[-0.499647801, -0.495221801, -0.490282801, -0...."
8,8,90,769.123284,"[-0.499320116, -0.497177116, -0.495045116, -0...."
9,9,144,779.269747,"[-0.499610204, -0.498010204, -0.495345204, -0...."


In [462]:
unitStruct_file = '/Users/guitchounts/Dropbox (coxlab)/Ephys/Data/GRat18/unitStruct.mat'

'/Users/guitchounts/Dropbox (coxlab)/Ephys/Data/GRat18/unitStruct.mat'

In [466]:
unitStruct['unitStruct'].dtype

dtype([('ratName', 'O'), ('ChGroup', 'O'), ('unitName', 'O'), ('chainNum', 'O'), ('chainEPhysFile', 'O'), ('cumchainNum', 'O'), ('spikeTimes', 'O'), ('spikeLabels', 'O'), ('startTime', 'O'), ('endTime', 'O'), ('quality', 'O'), ('wv', 'O')])

In [494]:
spikeTimes = unitStruct['unitStruct']['spikeTimes']
spikeLabels = unitStruct['unitStruct']['spikeLabels']

In [483]:
len(unitStruct['unitStruct'][0])

23

In [496]:
spikeTimes_clust6 = spikeTimes[0][5] ## times in samples from start of ephys file.
spikeLabels_clust6 = spikeLabels[0][5]

In [485]:
# loop over all clusters to find which ones have spikes from the ephys file we want, 636151800793559606

for clus in range(len(unitStruct['unitStruct'][0])):
    print clus
    
    # loop over the chainEPhysFile -- these say which files contributed to the cluster
    # find cluster that came from our file of interest
    

str(ephysfilename)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


'636151800793559606'

In [499]:
spikeTimes_clust6_from9606 = spikeTimes_clust6[np.where(spikeLabels_clust6==1)]

In [501]:
spikeTimes_clust6_from9606 = spikeTimes_clust6_from9606 / 3e4 ## convert to seconds from start of ephys file! 

In [502]:
spikeTimes_clust6_from9606

array([ 2705.2993    ,  2706.4576    ,  2711.47163333,  2715.0675    ,
        2715.33176667,  2715.3794    ,  2740.62206667,  2752.53043333,
        2753.1773    ,  2753.1953    ,  2758.999     ,  2759.2586    ,
        2759.84016667,  2760.0171    ,  2763.24993333,  2763.36806667,
        2771.08473333,  2776.87286667,  2780.90756667,  2780.98126667,
        2781.05773333,  2781.17136667,  2790.91503333,  2807.89733333,
        2807.92216667,  2808.95433333,  2809.1372    ,  2811.69823333,
        2822.12113333,  2822.1291    ,  2826.82586667,  2828.64393333,
        2829.0451    ,  2845.58576667,  2849.0658    ,  2852.60113333,
        2855.57686667,  2855.85033333,  2856.04483333,  2856.41816667,
        2856.56786667,  2856.61223333,  2857.50503333,  2857.64323333,
        2863.04233333,  2868.27996667,  2869.32546667,  2869.63006667,
        2869.7646    ,  2869.9981    ,  2870.53083333,  2871.8401    ,
        2871.9035    ,  2873.85046667,  2874.7885    ,  2874.79343333,
      

In [503]:
len(spikeTimes_clust6_from9606)

533

In [508]:
os.getcwd()

'/Users/guitchounts/Dropbox (coxlab)/Scripts/Repositories/continuous-ephys/titan_spikes_ttl'

In [509]:
ephys_file = '636152664381217973'
stim_data_7973 = pd.read_csv('~/Dropbox (coxlab)/Ephys/Data/GRat18/636152664381217973/oe_stim_times.csv') # read the CSV file with stimulus times and orientations

In [510]:
stim_data_7973

,Unnamed: 0,orientations,times
0,0,126,72973.225143
1,1,126,72997.600068
2,2,90,73006.013745
3,3,72,73008.263001
4,4,0,73024.090882
5,5,54,73024.624030
6,6,90,73039.702079
7,7,108,73069.258503
8,8,54,73138.101310
9,9,162,73149.747311


In [507]:
###### now that we have the times of the spikes in each cluster (46 clusters), 
###### let's loop over the behavior trials and each cluster to see if any cluster has spikes within the trial range

# make variable 'sorted_spikes' num_trials x 1. 

# for each behavior trial, add dictionary to sorted_spikes with name=cluster_number and entries = spike times.
trial_range = 0.5
rel_cluster_time = dict()
#sorted_spikes = []

for trial in trials:
    #trial=0
    trial_time = stim_data.times[trial] # the time of the visual stimulus on this trial. 

    print 'Trial %d ranges from %5.3f to %5.3f seconds' % (trial, trial_time-trial_range, trial_time+trial_range)

    
    temp_idx = (spikeTimes_clust6_from9606 < trial_time+trial_range) & (spikeTimes_clust6_from9606 > trial_time - trial_range)
    
    

    #print temp_idx

    temp_trial = spikeTimes_clust6_from9606[temp_idx] - trial_time ## subtract trial time so the spike time is in [-0.5 to +0.5]

    print temp_trial
    # rel_cluster_time will be our dict with keys=clustername, entries=times for the spike for this trial. 
    #rel_cluster_time[str(clusters['behavior_clusters']['num'][cluster][0][0][0])] = temp_trial

#sorted_spikes.append(rel_cluster_time)


Trial 0 ranges from 503.249 to 504.249 seconds
[]
Trial 1 ranges from 530.473 to 531.473 seconds
[]
Trial 2 ranges from 577.223 to 578.223 seconds
[]
Trial 3 ranges from 587.953 to 588.953 seconds
[]
Trial 4 ranges from 608.562 to 609.562 seconds
[]
Trial 5 ranges from 629.472 to 630.472 seconds
[]
Trial 6 ranges from 640.418 to 641.418 seconds
[]
Trial 7 ranges from 649.931 to 650.931 seconds
[]
Trial 8 ranges from 768.623 to 769.623 seconds
[]
Trial 9 ranges from 778.770 to 779.770 seconds
[]
Trial 10 ranges from 792.032 to 793.032 seconds
[]
Trial 11 ranges from 801.145 to 802.145 seconds
[]
Trial 12 ranges from 809.592 to 810.592 seconds
[]
Trial 13 ranges from 818.406 to 819.406 seconds
[]
Trial 14 ranges from 828.269 to 829.269 seconds
[]
Trial 15 ranges from 838.199 to 839.199 seconds
[]
Trial 16 ranges from 847.712 to 848.712 seconds
[]
Trial 17 ranges from 856.176 to 857.176 seconds
[]
Trial 18 ranges from 985.848 to 986.848 seconds
[]
Trial 19 ranges from 1224.199 to 1225.199

In [506]:
temp_trial

array([], dtype=float64)